### This code extracts data about 1) spike counts for each trial, separated into correct/incorrect contralateral, ipsilateral, and nogo 2) percent corrent choices for each session of behavior and 3) average time from stimulus presentation to auditory GO cue.

In [2]:
## to run this code, you need to have the behavioral data dictionaries previously loaded in
## see 'loading_in_tar' and 'loading_in_npy' files in repo, limited code copied below:
import os
import numpy as np
import pickle

vis_ctx = np.load("V1_spikes.npy", allow_pickle=True)
spikes_times = np.load('spikes_times.npy', allow_pickle=True)
goCue_times = np.load("goCue_times.npy", allow_pickle=True)
response_choices = np.load('response_choices.npy', allow_pickle=True)
feedback_types = np.load('feedback_types.npy', allow_pickle=True)
stim_times = np.load('stim_times.npy', allow_pickle=True)
vis_clusters = np.load('V1_clusters.npy', allow_pickle=True)
vis_timestamps = np.load('V1_timestamps.npy', allow_pickle=True)

In [31]:
## get trial types (contra/ipsi/nogo) for a given session
## calculate percent correct for each session

perc_corr = np.zeros(40)
sessions = np.arange(1, 40)
trial_type = {new_key: [] for new_key in sessions}

contra_corr_clusters = []
contra_corr_spiketimes = []
contra_corr_starts = []
contra_corr_ends = []

contra_incorr_clusters = []
contra_incorr_spiketimes = []
contra_incorr_starts = []
contra_incorr_ends = []

ipsi_corr_clusters = []
ipsi_corr_spiketimes = []
ipsi_corr_starts = []
ipsi_corr_ends = []

ipsi_incorr_clusters = []
ipsi_incorr_spiketimes = []
ipsi_incorr_starts = []
ipsi_incorr_ends = []

nogo_clusters = []
nogo_spiketimes = []
nogo_starts = []
nogo_ends = []

contra_starts = []
contra_sessions = []
ipsi_starts = []
ipsi_sessions = []
beh_contra = []
beh_ipsi = []

for s in sessions:
    print('session', s)
    trials = len(goCue_times[s]) # nTrials
    contra_Ctrials = np.zeros(trials) # tracking trials where correct response is contraleteral to probe
    contra_Itrials = np.zeros(trials) # tracking trials where INcorrect response is contraleteral to probe
    ipsi_Ctrials = np.zeros(trials) # tracking trials where correct response is ipsileteral to probe
    ipsi_Itrials = np.zeros(trials) # tracking trials where correct response is ipsileteral to probe
    nogo_trials = np.zeros(trials) # tracking NoGo trials
    
    for t in range(trials):
        response = int(response_choices[s][t]) # -1 right, 0 nogo, 1 left
        feedback = int(feedback_types[s][t]) # -1 incorrect, 1 correct
        if response == 1 and feedback == 1:
            contra_Ctrials[t] = 1
            contra_starts.append(stim_times[s][t])
            contra_sessions.append(s)
            beh_contra.append(1)
        elif response == 1 and feedback == -1:
            contra_Itrials[t] = 1
            beh_contra.append(0)
            contra_starts.append(stim_times[s][t])
            contra_sessions.append(s)
        elif response == -1 and feedback == 1:
            ipsi_Ctrials[t] = 1
            beh_ipsi.append(1)
            ipsi_starts.append(stim_times[s][t])
            ipsi_sessions.append(s)
        elif response == -1 and feedback == -1:
            ipsi_Itrials[t] = 1
            beh_ipsi.append(0)
            ipsi_starts.append(stim_times[s][t])
            ipsi_sessions.append(s)
        elif response == 0:
            nogo_trials[t] = 1
    
    pc = np.count_nonzero(feedback_types[s]==1)/(len(feedback_types[s]))
    perc_corr[s] = pc
    trial_type[s] = np.stack((contra_Ctrials, contra_Itrials, ipsi_Ctrials, ipsi_Itrials, nogo_trials), axis=0)
    
    ## now that we know which trials are which type, we need timestamps for each of those trials to get FR
    time_window = []
    
    for t in range(trials):

        lowerT = stim_times[s][t]
        upperT = goCue_times[s][t]
        this_trial_spikes = []
        this_cluster = []
        trial_start = []
        trial_end = []
        
        for key,value in vis_timestamps.items():
            if s == key:
                all_stamps = np.arange(1, len(vis_timestamps[s]))
                for i in all_stamps:
                    if lowerT <= vis_timestamps[s][i] < upperT: 
                        this_trial_spikes.append(float(vis_timestamps[s][i]))
                        this_cluster.append(int((vis_clusters[s][i])))
                        trial_start.append(lowerT)
                        trial_end.append(upperT)
                        
        time_window.append(upperT-lowerT)

        if trial_type[s][0][t] == 1: ## if this contra trial was correct
            contra_corr_spiketimes.extend(this_trial_spikes)
            contra_corr_clusters.extend(this_cluster)
            contra_corr_starts.extend(trial_start)
            contra_corr_ends.extend(trial_end)

        elif trial_type[s][1][t] == 1: ## if this contra trial was incorrect
            contra_incorr_spiketimes.extend(this_trial_spikes)
            contra_incorr_clusters.extend(this_cluster)
            contra_incorr_starts.extend(trial_start)
            contra_incorr_ends.extend(trial_end)

        elif trial_type[s][2][t] == 1: ## if this ipsi trial was correct
            ipsi_corr_spiketimes.extend(this_trial_spikes)
            ipsi_corr_clusters.extend(this_cluster)
            ipsi_corr_starts.extend(trial_start)
            ipsi_corr_ends.extend(trial_end)

        elif trial_type[s][3][t] == 1: ## if this ipsi trial was incorrect
            ipsi_incorr_spiketimes.extend(this_trial_spikes)
            ipsi_incorr_clusters.extend(this_cluster)
            ipsi_incorr_starts.extend(trial_start)
            ipsi_incorr_ends.extend(trial_end)

        elif trial_type[s][4][t] == 1: ## if this was a nogo trial
            nogo_spiketimes.extend(this_trial_spikes)
            nogo_clusters.extend(this_cluster)
            nogo_starts.extend(trial_start)
            nogo_ends.extend(trial_end)
            
contralateral_behavior = np.vstack((np.squeeze(np.asarray(contra_starts)), np.asarray(contra_sessions), 
                              np.asarray(beh_contra)))

ipsilateral_behavior = np.vstack((np.squeeze(np.asarray(ipsi_starts)), np.asarray(ipsi_sessions), 
                              np.asarray(beh_ipsi)))

np.savetxt('contralateral_behavior.txt', contralateral_behavior)
np.savetxt('ipsilateral_behavior.txt', ipsilateral_behavior)

In [23]:
contra_corr_dict = {'n_neuron':contra_corr_clusters, 'spiketimes':contra_corr_spiketimes, 'starts': contra_corr_starts, 'ends': contra_corr_ends}
contra_incorr_dict = {'n_neuron':contra_incorr_clusters, 'spiketimes':contra_incorr_spiketimes, 'starts': contra_incorr_starts, 'ends': contra_incorr_ends}
ipsi_corr_dict = {'n_neuron':ipsi_corr_clusters, 'spiketimes':ipsi_corr_spiketimes, 'starts': ipsi_corr_starts, 'ends': ipsi_corr_ends}
ipsi_incorr_dict = {'n_neuron':ipsi_incorr_clusters, 'spiketimes':ipsi_incorr_spiketimes, 'starts': ipsi_incorr_starts, 'ends': ipsi_incorr_ends}
nogo_dict = {'n_neuron':nogo_clusters, 'spiketimes':nogo_spiketimes, 'starts': nogo_starts, 'ends': nogo_ends}

os.chdir('/Users/srbindas/Documents/GitHub/NMA2020-Seals-Evading-Hidden-States/npy files')

file_to_write = open("contra_corr_dict.npy", "wb")
pickle.dump(contra_corr_dict, file_to_write)

file_to_write = open("contra_incorr_dict.npy", "wb")
pickle.dump(contra_incorr_dict, file_to_write)

file_to_write = open("ipsi_corr_dict.npy", "wb")
pickle.dump(ipsi_corr_dict, file_to_write)

file_to_write = open("ipsi_incorr_dict.npy", "wb")
pickle.dump(ipsi_incorr_dict, file_to_write)

file_to_write = open("nogo_dict.npy", "wb")
pickle.dump(nogo_dict, file_to_write)

In [14]:
## FOR PCA
nrows = len(contra_corr_dict['n_neuron'])+len(contra_incorr_dict['n_neuron'])
for_pca = np.zeros((nrows, 4))
for_pca[0:(len(contra_corr_dict['n_neuron'])), 0]=contra_corr_dict['n_neuron']
for_pca[0:(len(contra_corr_dict['n_neuron'])), 1]=contra_corr_dict['spiketimes']
for_pca[0:(len(contra_corr_dict['n_neuron'])), 2]=contra_corr_dict['starts']
for_pca[0:(len(contra_corr_dict['n_neuron'])), 3]=contra_corr_dict['ends']
for_pca[len(contra_corr_dict['n_neuron']):, 0]=contra_incorr_dict['n_neuron']
for_pca[len(contra_corr_dict['n_neuron']):, 1]=contra_incorr_dict['spiketimes']
for_pca[len(contra_corr_dict['n_neuron']):, 2]=contra_incorr_dict['starts']
for_pca[len(contra_corr_dict['n_neuron']):, 3]=contra_incorr_dict['ends']

In [24]:
# saving to file
np.savetxt('matrix_for_pca.txt', for_pca)

In [17]:
## FOR PCA - validation
nrows = len(ipsi_corr_dict['n_neuron'])+len(ipsi_incorr_dict['n_neuron'])
for_pca_IPS = np.zeros((nrows, 4))
for_pca_IPS[0:(len(ipsi_corr_dict['n_neuron'])), 0]=ipsi_corr_dict['n_neuron']
for_pca_IPS[0:(len(ipsi_corr_dict['n_neuron'])), 1]=ipsi_corr_dict['spiketimes']
for_pca_IPS[0:(len(ipsi_corr_dict['n_neuron'])), 2]=ipsi_corr_dict['starts']
for_pca_IPS[0:(len(ipsi_corr_dict['n_neuron'])), 3]=ipsi_corr_dict['ends']
for_pca_IPS[len(ipsi_corr_dict['n_neuron']):, 0]=ipsi_incorr_dict['n_neuron']
for_pca_IPS[len(ipsi_corr_dict['n_neuron']):, 1]=ipsi_incorr_dict['spiketimes']
for_pca_IPS[len(ipsi_corr_dict['n_neuron']):, 2]=ipsi_incorr_dict['starts']
for_pca_IPS[len(ipsi_corr_dict['n_neuron']):, 3]=ipsi_incorr_dict['ends']

In [25]:
# saving to file
np.savetxt('ipsilateral_for_pca.txt', for_pca_IPS)